# Tabular

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
from PIL import Image
import time
import os
import copy
from collections import Counter

In [2]:
tabular_data = "../../data/SkidSteer_2019-08.csv"

df = pd.read_csv(tabular_data, index_col=1)
df['Unique_ID'] = df[['Source','item#']].apply(lambda x: '_'.join(x),axis = 1)
df = df.filter(['Unique_ID','Winning Bid','Hours Final','Age at Sale (bin)','Bucket','Engine','Tires','Transmission'], axis = 1)
df = df.rename(columns={
    'Unique_ID': "unique_id",
    'Hours Final': "hours_final",
    'Winning Bid': "winning_bid",
    'Age at Sale (bin)': "age_at_sale",
    'Bucket': "bucket",
    'Engine': "engine",
    'Tires': "tires",
    'Transmission': "transmission"
})
# color = pd.read_csv('skid_steer_color_score.csv')
# final_df = pd.merge(new_df, color,on='Unique_ID',how='inner')


### removal
# remove duplicant
duplicated_item = [item for item, count in Counter(df["unique_id"]).items() if count > 1]
df = df[~df['unique_id'].isin(duplicated_item)]

# remove not matched rows
image_item = [img_name.strip(".jpg") for img_name in os.listdir("../../data/images/")]
df = df[df["unique_id"].isin(image_item)]

# remove comma
df["winning_bid"] = df["winning_bid"].str.replace(',', '').astype(int)

# remove special image
df = df[df['unique_id'] != "rbauction_10525632"]

In [3]:
### winning_bid

# log-transform
df["winning_bid"] = np.log(df["winning_bid"])

# min max scale
mm_scaler_price = preprocessing.MinMaxScaler((-1, 1))
df["winning_bid"] = mm_scaler_price.fit_transform(df["winning_bid"].to_numpy().reshape(-1, 1))

In [4]:
### hours_final

# impute nan with median and new binary indicator
df["hours_final"] = df["hours_final"].str.replace(",", "")
df["hours_final"] = df["hours_final"].astype(float)
df.insert(3, column="hours_final_nan", value=df["hours_final"].isna().astype(int))
df.loc[df["hours_final"].isna(), "hours_final"] = df["hours_final"].median(skipna=True)

# log transform
df["hours_final"] = np.log(df["hours_final"])

# normalize
rb_scaler_hour = preprocessing.RobustScaler()
df["hours_final"] = rb_scaler_hour.fit_transform(np.array(df["hours_final"]).reshape(-1, 1))

In [5]:
### age_at_sale

# impute nan with median and new binary indicator
df["age_at_sale"] = df["age_at_sale"].astype(float)
df.insert(5, column="age_at_sale_nan", value=df["age_at_sale"].isna().astype(int))
df.loc[df["age_at_sale"].isna(), "age_at_sale"] = df["age_at_sale"].median(skipna=True)

# normalize
rb_scaler_age = preprocessing.RobustScaler()
df["age_at_sale"] = rb_scaler_age.fit_transform(np.array(df["age_at_sale"]).reshape(-1, 1))

In [6]:
### bucket
df.insert(7, column="bucket_bin", value=0)
df.loc[
    ~df["bucket"].isna() & 
    df["bucket"].str.contains("bucket", case=False) | 
    df["bucket"].str.contains("bkt", case=False), "bucket_bin"
] = 1

In [7]:
# shuffle and split
np.random.seed(1)
split = [0.7, 0.3]
split0 = round(df.shape[0] * split[0])
# split1 = round(df.shape[0] * (split[0] + split[1]))
df = df.sample(frac=1)
df_train = df.iloc[:split0]
df_val = df.iloc[split0:]

In [8]:
df_train.head()

,unique_id,winning_bid,hours_final,hours_final_nan,age_at_sale,age_at_sale_nan,bucket,bucket_bin,engine,tires,transmission
Item Id,,,,,,,,,,,
ec1907ce-5c12-5ab8-b42d-39eb481e6049,ironplanet_1703726,0.105406,-6.732824,0,-0.666667,0,"74"" Wide General Purpose Smooth Edge Bucket",1,NaN,Cushion Tires,NaN
529071,PW_DD1289,0.172815,-1.629216,0,-1.000000,0,"Kubota 68""W bucket",1,Kubota V3307-CR four cylinder turbo diesel engine,Titan 12-16.5 NHS tires,Two speed hydrostatic transmission
eff85cf8-4988-8990-b317-39e91a64ec3a,rbauction_10239624,0.219707,-0.164164,0,-0.666667,0,hyd Q/C bkt,1,NaN,NaN,NaN
386158,PW_H1380,-0.072566,-0.769452,0,-0.333333,0,"80""W bucket",1,"81 HP, Case 3.2L four cylinder turbo diesel en...",12-16.5 tires,Hydrostatic transmission
460813,PW_J8873,-0.305592,-0.416323,0,-0.333333,0,NaN,0,John Deere 5030TT001 3.0L turbo diesel engine,12-16.5 tires,Two speed hydrostatic transmission


In [9]:
df_val.head()

,unique_id,winning_bid,hours_final,hours_final_nan,age_at_sale,age_at_sale_nan,bucket,bucket_bin,engine,tires,transmission
Item Id,,,,,,,,,,,
cdf26f3a-f792-7a0f-350e-39e9461c54c9,bigiron_EN9531,0.164709,-0.503074,0,-0.333333,0,"84"" Bucket",1,"4-Cyl Turbo Diesel Engine, 84 Hp","Some Tires Have Cuts (Pictured), Tires- 14-17....",NaN
14c666d4-512a-8962-6072-39e918e5b2f7,rbauction_10693873,-0.861828,-0.443163,0,1.333333,0,bkt,1,NaN,NaN,NaN
dc20ac44-7c35-11a7-2dfa-39e8cc01c274,bigiron_BO0203,-0.180158,-0.226316,0,1.333333,0,"Buckets 12"", 64"" Bucket",1,58 Horse Power Isuzu Diesel Engine,"12-16.5 Tires, Spare Tire And Rim",Hydrostat Transmission
ebfb5935-40e0-4488-2e6c-39ea98658096,ironplanet_1891010,-0.402470,-6.732824,0,-0.333333,0,"72"" Wide General Purpose Smooth Edge Bucket",1,NaN,NaN,NaN
446849,PW_J2936,-0.402470,-1.926828,0,-0.333333,0,NaN,0,"Case four cylinder turbo diesel engine, Non-op...",12-16.5 tires,Hydrostatic transmission


In [10]:
df_train.to_csv("./SkidSteer_2019-08_clean_train.csv")
df_val.to_csv("./SkidSteer_2019-08_clean_val.csv")

# Torch

In [11]:
class skidsteer_dataset(Dataset):
    """Corrosion Detection dataset."""

    def __init__(self, 
                 csv_file, 
                 img_root, 
                 transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            img_root (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample (including augmentation).
        """
        self.csv_file = pd.read_csv(csv_file, index_col=0)
        self.img_root = img_root
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        '''Return one data point with a PIL image and its label.'''
        img_dir = os.path.join(self.img_root, self.csv_file["unique_id"][idx]) + ".jpg"
        price = self.csv_file["winning_bid"][idx]
        image = Image.open(img_dir)
        others = torch.tensor(self.csv_file.iloc[idx, [2, 3, 4, 5, 7]])
        
        if self.transform:
            image = self.transform(image)
        sample = {'image': image, 'price': price, "others": others}
        return sample

In [12]:
def norm2price(tensor, min_max_scaler):
    array2d = tensor.to("cpu").data.numpy().reshape(-1, 1)
    return np.exp(min_max_scaler.inverse_transform(array2d))

def price_MAE(outputs, prices, min_max_scaler):
    outputs = norm2price(outputs, min_max_scaler)
    prices = norm2price(prices, min_max_scaler)
    return np.abs(outputs - prices).mean()

In [13]:
def train_model(model, model_LU, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch = None
    best_loss = float("Inf")
    best_mae = float("Inf")

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_mae = 0.0
            
            # Iterate over data.
            for items in dataloaders[phase]:
                images = items["image"].to(device)
                prices = items["price"].to(device)
                others = items["others"].to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = cat_net(model, model_LU, images, others).squeeze()
                    loss = criterion(outputs, prices)
                    mae = price_MAE(outputs, prices, mm_scaler_price)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * images.size(0)
                running_mae += mae * images.size(0)
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_mae = running_mae / dataset_sizes[phase]
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))
            print('{} MAE: {:.4f}'.format(phase, epoch_mae))
            
            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_epoch = epoch + 1
                best_loss = epoch_loss
                best_mae = epoch_mae
                best_model_wts = copy.deepcopy(model.state_dict())
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f} at epoch {}'.format(best_loss, best_epoch))
    print('Best val MAE: {:4f} at epoch {}'.format(best_mae, best_epoch))

    # load best model weights
    print("\nLoad the model weights at the best epoch")
    model.load_state_dict(best_model_wts)
    return model

In [14]:
CSV_FILE = {"train": "./SkidSteer_2019-08_clean_train.csv",
            "val": "./SkidSteer_2019-08_clean_val.csv"}
IMG_ROOT = "../../data/images/"
TRANSFORM = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

datasets = {x: skidsteer_dataset(csv_file=CSV_FILE[x],
                                 img_root=IMG_ROOT,
                                 transform=TRANSFORM[x])
            for x in ["train", "val"]}
dataloaders = {x: DataLoader(datasets[x], 
                             batch_size=4, 
                             shuffle=True, 
                             num_workers=4)
               for x in ["train", "val"]}
dataset_sizes = {x: len(datasets[x]) for x in ["train", "val"]}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## ResNet18 - Finetuning Pretrained

In [15]:
def cat_net(model, model_LU, images, others):
    return model(images)

In [16]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, 1), nn.Tanh())

### LU
model_LU = nn.Identity()

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [17]:
model_res18_image = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.2362
train MAE: 7432.2630
val Loss: 0.1074
val MAE: 4514.8336

Epoch 2/50
----------
train Loss: 0.2508
train MAE: 7077.3206
val Loss: 0.0776
val MAE: 4065.6795

Epoch 3/50
----------
train Loss: 0.1742
train MAE: 5546.5274
val Loss: 0.2050
val MAE: 5287.5337

Epoch 4/50
----------
train Loss: 0.1766
train MAE: 5430.9378
val Loss: 0.0848
val MAE: 3975.4101

Epoch 5/50
----------
train Loss: 0.1683
train MAE: 5358.1926
val Loss: 0.1757
val MAE: 5027.0539

Epoch 6/50
----------
train Loss: 0.1590
train MAE: 5044.5820
val Loss: 0.0880
val MAE: 4140.0839

Epoch 7/50
----------
train Loss: 0.1783
train MAE: 5312.3470
val Loss: 0.0998
val MAE: 4213.8722

Epoch 8/50
----------
train Loss: 0.1618
train MAE: 5109.9032
val Loss: 0.1178
val MAE: 4215.8714

Epoch 9/50
----------
train Loss: 0.1670
train MAE: 5092.6403
val Loss: 0.1736
val MAE: 4796.5179

Epoch 10/50
----------
train Loss: 0.1784
train MAE: 5290.8680
val Loss: 0.1150
val MAE: 5379.8446

Epoch 11/

## ResNet18 - Finetuning Pretrained + All

In [12]:
def cat_net(model, model_LU, images, others):
    z = torch.cat((model(images), others), dim=1)
    return model_LU(z)

In [13]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1), 
    nn.Tanh()
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [14]:
model_res18_baseline = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.1356
train MAE: 5614.8389
val Loss: 0.1141
val MAE: 5614.9234

Epoch 2/50
----------
train Loss: 0.1037
train MAE: 4790.4702
val Loss: 0.0758
val MAE: 4032.3060

Epoch 3/50
----------
train Loss: 0.0944
train MAE: 4584.3763
val Loss: 0.0692
val MAE: 3972.2783

Epoch 4/50
----------
train Loss: 0.0928
train MAE: 4540.2998
val Loss: 0.0717
val MAE: 4143.0648

Epoch 5/50
----------
train Loss: 0.0837
train MAE: 4326.6119
val Loss: 0.0583
val MAE: 3574.8249

Epoch 6/50
----------
train Loss: 0.0820
train MAE: 4280.3547
val Loss: 0.0630
val MAE: 3825.1179

Epoch 7/50
----------
train Loss: 0.0775
train MAE: 4177.0761
val Loss: 0.0533
val MAE: 3427.6889

Epoch 8/50
----------
train Loss: 0.0763
train MAE: 4095.6283
val Loss: 0.0563
val MAE: 3438.6908

Epoch 9/50
----------
train Loss: 0.0721
train MAE: 4021.6277
val Loss: 0.0542
val MAE: 3423.3054

Epoch 10/50
----------
train Loss: 0.0680
train MAE: 3897.3020
val Loss: 0.0551
val MAE: 3530.8266

Epoch 11/

## ResNet18 - Finetuning Pretrained + All + 128

In [18]:
def cat_net(model, model_LU, images, others):
    z = torch.cat((model(images), others), dim=1)
    return model_LU(z)

In [19]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 128), 
    nn.Tanh(),
    nn.Linear(128, 1), 
    nn.Tanh()
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [20]:
model_res18_128 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.1412
train MAE: 5939.8839
val Loss: 0.0885
val MAE: 4758.9123

Epoch 2/50
----------
train Loss: 0.0838
train MAE: 4341.5449
val Loss: 0.0596
val MAE: 3723.5275

Epoch 3/50
----------
train Loss: 0.0766
train MAE: 4118.7780
val Loss: 0.0514
val MAE: 3333.6819

Epoch 4/50
----------
train Loss: 0.0739
train MAE: 4034.0997
val Loss: 0.0468
val MAE: 3212.5126

Epoch 5/50
----------
train Loss: 0.0713
train MAE: 3971.9295
val Loss: 0.0455
val MAE: 3137.6729

Epoch 6/50
----------
train Loss: 0.0660
train MAE: 3840.5641
val Loss: 0.0435
val MAE: 3051.4690

Epoch 7/50
----------
train Loss: 0.0661
train MAE: 3812.0188
val Loss: 0.0437
val MAE: 3078.3816

Epoch 8/50
----------
train Loss: 0.0631
train MAE: 3736.3789
val Loss: 0.0421
val MAE: 3020.5392

Epoch 9/50
----------
train Loss: 0.0616
train MAE: 3689.9969
val Loss: 0.0437
val MAE: 3080.3123

Epoch 10/50
----------
train Loss: 0.0594
train MAE: 3635.7954
val Loss: 0.0429
val MAE: 3029.2892

Epoch 11/

## ResNet18 - Finetuning Pretrained + All + 128/32

In [28]:
def cat_net(model, model_LU, images, others):
    z = torch.cat((model(images), others), dim=1)
    return model_LU(z)

In [29]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 128), 
    nn.Tanh(),
    nn.Linear(128, 32), 
    nn.Tanh(),
    nn.Linear(32, 1), 
    nn.Tanh()
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)

In [30]:
model_res18_128_32 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)

Epoch 1/100
----------
train Loss: 0.1159
train MAE: 5135.7053
val Loss: 0.0716
val MAE: 4033.9209

Epoch 2/100
----------
train Loss: 0.0877
train MAE: 4449.4023
val Loss: 0.0527
val MAE: 3399.2730

Epoch 3/100
----------
train Loss: 0.0798
train MAE: 4238.6564
val Loss: 0.0527
val MAE: 3395.1475

Epoch 4/100
----------
train Loss: 0.0749
train MAE: 4100.4653
val Loss: 0.0461
val MAE: 3147.5430

Epoch 5/100
----------
train Loss: 0.0727
train MAE: 4017.0093
val Loss: 0.0456
val MAE: 3123.5580

Epoch 6/100
----------
train Loss: 0.0721
train MAE: 3999.9321
val Loss: 0.0459
val MAE: 3147.9431

Epoch 7/100
----------
train Loss: 0.0690
train MAE: 3912.8747
val Loss: 0.0474
val MAE: 3191.2375

Epoch 8/100
----------
train Loss: 0.0671
train MAE: 3884.4400
val Loss: 0.0480
val MAE: 3251.5913

Epoch 9/100
----------
train Loss: 0.0655
train MAE: 3825.2709
val Loss: 0.0446
val MAE: 3072.4387

Epoch 10/100
----------
train Loss: 0.0638
train MAE: 3760.2155
val Loss: 0.0418
val MAE: 2967.5012


## ResNet18 - Finetuning Pretrained + All + 64

In [24]:
def cat_net(model, model_LU, images, others):
    z = torch.cat((model(images), others), dim=1)
    return model_LU(z)

In [26]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 64), 
    nn.Tanh(),
    nn.Linear(64, 1), 
    nn.Tanh()
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [27]:
model_res18_64 = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.1162
train MAE: 5196.6801
val Loss: 0.0746
val MAE: 4206.9782

Epoch 2/50
----------
train Loss: 0.0806
train MAE: 4239.7448
val Loss: 0.0517
val MAE: 3327.6908

Epoch 3/50
----------
train Loss: 0.0733
train MAE: 4039.7859
val Loss: 0.0493
val MAE: 3201.2275

Epoch 4/50
----------
train Loss: 0.0731
train MAE: 4040.3184
val Loss: 0.0475
val MAE: 3149.6006

Epoch 5/50
----------
train Loss: 0.0686
train MAE: 3923.4169
val Loss: 0.0481
val MAE: 3201.4702

Epoch 6/50
----------
train Loss: 0.0662
train MAE: 3810.0108
val Loss: 0.0449
val MAE: 3096.5315

Epoch 7/50
----------
train Loss: 0.0641
train MAE: 3764.3894
val Loss: 0.0438
val MAE: 3043.7834

Epoch 8/50
----------
train Loss: 0.0600
train MAE: 3666.5070
val Loss: 0.0436
val MAE: 3034.1040

Epoch 9/50
----------
train Loss: 0.0618
train MAE: 3692.2256
val Loss: 0.0447
val MAE: 3108.9914

Epoch 10/50
----------
train Loss: 0.0619
train MAE: 3704.5725
val Loss: 0.0420
val MAE: 3002.8704

Epoch 11/

## ResNet50 - Finetuning Pretrained + All

In [45]:
def cat_net(model, model_LU, images, others):
    z = torch.cat((model(images), others), dim=1)
    return model_LU(z)

In [46]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1), 
    nn.Tanh()
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [47]:
model_res50_baseline = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.1079
train MAE: 5027.4760
val Loss: 0.0540
val MAE: 3449.4739

Epoch 2/50
----------
train Loss: 0.0715
train MAE: 4027.1676
val Loss: 0.0552
val MAE: 3483.5180

Epoch 3/50
----------
train Loss: 0.0661
train MAE: 3842.1879
val Loss: 0.0555
val MAE: 3487.4897

Epoch 4/50
----------
train Loss: 0.0624
train MAE: 3713.8836
val Loss: 0.0447
val MAE: 3093.7736

Epoch 5/50
----------
train Loss: 0.0584
train MAE: 3604.2721
val Loss: 0.0484
val MAE: 3217.8245

Epoch 6/50
----------
train Loss: 0.0554
train MAE: 3502.6363
val Loss: 0.0495
val MAE: 3292.5552

Epoch 7/50
----------
train Loss: 0.0507
train MAE: 3355.7993
val Loss: 0.0478
val MAE: 3186.8968

Epoch 8/50
----------
train Loss: 0.0493
train MAE: 3289.5496
val Loss: 0.0397
val MAE: 2876.6735

Epoch 9/50
----------
train Loss: 0.0487
train MAE: 3266.1292
val Loss: 0.0400
val MAE: 2878.9057

Epoch 10/50
----------
train Loss: 0.0463
train MAE: 3198.7546
val Loss: 0.0413
val MAE: 2927.4577

Epoch 11/

## ResNet152 - Finetuning Pretrained + All

In [56]:
def cat_net(model, model_LU, images, others):
    z = torch.cat((model(images), others), dim=1)
    return model_LU(z)

In [57]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1), 
    nn.Tanh()
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [58]:
model_res152_baseline = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0921
train MAE: 4558.7505
val Loss: 0.0535
val MAE: 3471.7156

Epoch 2/50
----------
train Loss: 0.0723
train MAE: 4022.9528
val Loss: 0.0502
val MAE: 3367.6602

Epoch 3/50
----------
train Loss: 0.0654
train MAE: 3837.9298
val Loss: 0.0512
val MAE: 3346.8981

Epoch 4/50
----------
train Loss: 0.0617
train MAE: 3730.4143
val Loss: 0.0430
val MAE: 2997.8525

Epoch 5/50
----------
train Loss: 0.0541
train MAE: 3452.0233
val Loss: 0.0466
val MAE: 3204.2803

Epoch 6/50
----------
train Loss: 0.0538
train MAE: 3445.2714
val Loss: 0.0419
val MAE: 2992.6813

Epoch 7/50
----------
train Loss: 0.0504
train MAE: 3317.8484
val Loss: 0.0409
val MAE: 2961.6367

Epoch 8/50
----------
train Loss: 0.0455
train MAE: 3143.1174
val Loss: 0.0394
val MAE: 2862.4014

Epoch 9/50
----------
train Loss: 0.0443
train MAE: 3123.8915
val Loss: 0.0439
val MAE: 3030.2953

Epoch 10/50
----------
train Loss: 0.0454
train MAE: 3116.0346
val Loss: 0.0386
val MAE: 2811.0698

Epoch 11/

## ResNeXt-50 - Finetuning Pretrained + All

In [15]:
def cat_net(model, model_LU, images, others):
    z = torch.cat((model(images), others), dim=1)
    return model_LU(z)

In [16]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnext50_32x4d(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1), 
    nn.Tanh()
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [17]:
model_inceptionv3_baseline = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0892
train MAE: 4484.9749
val Loss: 0.0580
val MAE: 3511.8422

Epoch 2/50
----------
train Loss: 0.0773
train MAE: 4128.6109
val Loss: 0.0591
val MAE: 3575.5944

Epoch 3/50
----------
train Loss: 0.0703
train MAE: 4003.4539
val Loss: 0.0668
val MAE: 3680.7229

Epoch 4/50
----------
train Loss: 0.0685
train MAE: 3864.3514
val Loss: 0.0472
val MAE: 3128.9136

Epoch 5/50
----------
train Loss: 0.0670
train MAE: 3853.9215
val Loss: 0.0524
val MAE: 3396.5823

Epoch 6/50
----------
train Loss: 0.0630
train MAE: 3713.7354
val Loss: 0.0428
val MAE: 2959.6967

Epoch 7/50
----------
train Loss: 0.0595
train MAE: 3611.8714
val Loss: 0.0520
val MAE: 3206.2538

Epoch 8/50
----------
train Loss: 0.0558
train MAE: 3491.7581
val Loss: 0.0519
val MAE: 3430.9789

Epoch 9/50
----------
train Loss: 0.0545
train MAE: 3448.4659
val Loss: 0.0462
val MAE: 3065.3481

Epoch 10/50
----------
train Loss: 0.0536
train MAE: 3424.8984
val Loss: 0.0447
val MAE: 3007.8446

Epoch 11/

## ResNeXt-101 - Finetuning Pretrained + All

In [18]:
def cat_net(model, model_LU, images, others):
    z = torch.cat((model(images), others), dim=1)
    return model_LU(z)

In [19]:
torch.manual_seed(0)

### ResNet18
model_ft = models.resnext101_32x8d(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Identity()

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 1), 
    nn.Tanh()
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /home/yifei/.cache/torch/checkpoints/resnext101_32x8d-8ba56ff5.pth
100%|██████████| 340M/340M [00:08<00:00, 41.7MB/s] 


In [20]:
model_inceptionv3_baseline = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.0816
train MAE: 4306.0882
val Loss: 0.0512
val MAE: 3313.3152

Epoch 2/50
----------
train Loss: 0.0605
train MAE: 3677.7450
val Loss: 0.0526
val MAE: 3356.6535

Epoch 3/50
----------
train Loss: 0.0527
train MAE: 3420.7240
val Loss: 0.0452
val MAE: 3181.6026

Epoch 4/50
----------
train Loss: 0.0473
train MAE: 3227.8587
val Loss: 0.0403
val MAE: 2964.0020

Epoch 5/50
----------
train Loss: 0.0446
train MAE: 3139.0384
val Loss: 0.0447
val MAE: 3138.5471

Epoch 6/50
----------
train Loss: 0.0411
train MAE: 2989.4788
val Loss: 0.0397
val MAE: 2864.7096

Epoch 7/50
----------
train Loss: 0.0398
train MAE: 2961.2079
val Loss: 0.0378
val MAE: 2760.0631

Epoch 8/50
----------
train Loss: 0.0362
train MAE: 2824.0654
val Loss: 0.0376
val MAE: 2830.2350

Epoch 9/50
----------
train Loss: 0.0362
train MAE: 2811.0657
val Loss: 0.0396
val MAE: 2937.7893

Epoch 10/50
----------
train Loss: 0.0332
train MAE: 2719.4101
val Loss: 0.0407
val MAE: 2918.2864

Epoch 11/

## VGG19 - Finetuning Pretrained + All

In [21]:
def cat_net(model, model_LU, images, others):
    z = torch.cat((model(images), others), dim=1)
    return model_LU(z)

In [22]:
torch.manual_seed(0)

### ResNet18
model_ft = models.vgg19_bn(pretrained=True)
num_ftrs = model_ft.classifier[0].in_features
model_ft.classifier = nn.Identity()

### LU
model_LU = nn.Sequential(
    nn.Linear(num_ftrs + 5, 4096), 
    nn.ReLU(), 
    nn.Dropout(p=0.5, inplace=False), 
    nn.Linear(4096, 64),
    nn.ReLU(), 
    nn.Dropout(p=0.5, inplace=False), 
    nn.Linear(64, 1)
)

### model setup
model_ft = model_ft.to(device)
model_LU = model_LU.to(device)
criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [23]:
model_vgg19_baseline = train_model(model_ft, model_LU, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.1441
train MAE: 5767.8434
val Loss: 0.1262
val MAE: 5249.6555

Epoch 2/50
----------
train Loss: 0.1268
train MAE: 5376.9578
val Loss: 0.1008
val MAE: 4643.1060

Epoch 3/50
----------
train Loss: 0.1125
train MAE: 5049.9502
val Loss: 0.0788
val MAE: 4093.2936

Epoch 4/50
----------
train Loss: 0.0995
train MAE: 4697.1607
val Loss: 0.0750
val MAE: 4061.7651

Epoch 5/50
----------
train Loss: 0.0890
train MAE: 4443.4585
val Loss: 0.0655
val MAE: 3794.0584

Epoch 6/50
----------
train Loss: 0.0873
train MAE: 4393.4708
val Loss: 0.0671
val MAE: 3951.5464

Epoch 7/50
----------
train Loss: 0.0834
train MAE: 4300.8714
val Loss: 0.0703
val MAE: 3913.3267

Epoch 8/50
----------
train Loss: 0.0780
train MAE: 4135.2584
val Loss: 0.0628
val MAE: 3807.0066

Epoch 9/50
----------
train Loss: 0.0770
train MAE: 4139.3307
val Loss: 0.0602
val MAE: 3519.3175

Epoch 10/50
----------
train Loss: 0.0738
train MAE: 4024.1717
val Loss: 0.0594
val MAE: 3586.2332

Epoch 11/